In [30]:
import requests
from bs4 import BeautifulSoup
from flask import current_app
import pandas as pd

In [31]:
with open('../static/data/seoul_api_key.txt', mode='r') as key_fd:
        seoulapi_key = key_fd.read(100)

In [32]:
corona_url = 'http://openapi.seoul.go.kr:8088'

In [33]:
type = 'xml'
url = f'{corona_url}/{seoulapi_key}/{type}/Corona19Status/1/10'
result = requests.get(url)

In [34]:
soup = BeautifulSoup(result.text, 'xml')
resultCode = soup.find('RESULT').find('CODE').get_text()

In [41]:
resultCode

'INFO-000'

In [44]:
list_total_count = int(soup.find('list_total_count').get_text())
list_total_count

22859

In [54]:
import sqlite3
def get_seoul_last_sid():
    conn = sqlite3.connect('../db/covid.db')
    cur = conn.cursor()

    sql = f'select sid from seoul order by sid desc limit 1;'
    cur.execute(sql)
    row = cur.fetchone()

    cur.close()
    conn.close()
    return row[0]

In [55]:
last_sid = get_seoul_last_sid()

In [61]:
read_count = list_total_count - last_sid

In [65]:
id_list, date_list, area_list, travel_list = [], [], [], []
contact_list, status_list, moving_list = [], [], []
for i in range(1, read_count, 1000):
    start_index = i
    end_index = i+1000-1 if i+1000-1 < read_count else read_count
    url = f'{corona_url}/{seoulapi_key}/{type}/Corona19Status/{start_index}/{end_index}'
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'xml')
    items = soup.find_all('row')

for item in items:
    id_list.append(item.find('CORONA19_ID').string)
    date_list.append(item.find('CORONA19_DATE').string)
    area_list.append(item.find('CORONA19_AREA').string)
    travel_list.append(item.find('CORONA19_TRAVEL_HISTORY').string)
    contact_list.append(item.find('CORONA19_CONTACT_HISTORY').string)
    status_list.append(item.find('CORONA19_LEAVE_STATUS').string)
    moving_list.append(item.find('CORONA19_MOVING_PATH').string)

In [68]:
df = pd.DataFrame({'연번': id_list, '확진일': date_list, '지역': area_list, '여행력': travel_list,'접촉력': contact_list, '상태': status_list, '이동경로': moving_list})
df['연번'] = df['연번'].astype(int)
df['연번']

0      18859
1      18858
2      18857
3      18856
4      18855
       ...  
601    18258
602    18257
603    18256
604    18255
605    18254
Name: 연번, Length: 606, dtype: int32

In [69]:
df.sort_values('연번', inplace=True)
df

,연번,확진일,지역,여행력,접촉력,상태,이동경로
605,18254,2020-12-29,동작구,None,타시도 확진자 접촉,퇴원,이동경로 공개기간 경과
604,18255,2020-12-29,동작구,None,감염경로 조사중,퇴원,이동경로 공개기간 경과
603,18256,2020-12-29,용산구,None,기타 확진자 접촉,퇴원,이동경로 공개기간 경과
602,18257,2020-12-29,도봉구,None,기타 확진자 접촉,퇴원,이동경로 공개기간 경과
601,18258,2020-12-29,동작구,None,감염경로 조사중,퇴원,이동경로 공개기간 경과
...,...,...,...,...,...,...,...
4,18855,2020-12-30,종로구,None,감염경로 조사중,사망,이동경로 공개기간 경과
3,18856,2020-12-30,기타,None,동부구치소 관련,None,이동경로 공개기간 경과
2,18857,2020-12-30,기타,None,동부구치소 관련,None,이동경로 공개기간 경과
1,18858,2020-12-30,중랑구,None,기타 확진자 접촉,퇴원,이동경로 공개기간 경과


In [83]:
df['확진일'][0]

'2020-12-30'

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 605 to 0
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      606 non-null    int32 
 1   확진일     606 non-null    object
 2   지역      606 non-null    object
 3   여행력     5 non-null      object
 4   접촉력     606 non-null    object
 5   상태      414 non-null    object
 6   이동경로    606 non-null    object
dtypes: int32(1), object(6)
memory usage: 35.5+ KB


In [86]:
df['확진일']

605    2020-12-29
604    2020-12-29
603    2020-12-29
602    2020-12-29
601    2020-12-29
          ...    
4      2020-12-30
3      2020-12-30
2      2020-12-30
1      2020-12-30
0      2020-12-30
Name: 확진일, Length: 606, dtype: object

In [90]:
df['확진일'].dtypes

dtype('O')

In [92]:
df.astype(str)

,연번,확진일,지역,여행력,접촉력,상태,이동경로
605,18254,2020-12-29,동작구,nan,타시도 확진자 접촉,퇴원,이동경로 공개기간 경과
604,18255,2020-12-29,동작구,nan,감염경로 조사중,퇴원,이동경로 공개기간 경과
603,18256,2020-12-29,용산구,nan,기타 확진자 접촉,퇴원,이동경로 공개기간 경과
602,18257,2020-12-29,도봉구,nan,기타 확진자 접촉,퇴원,이동경로 공개기간 경과
601,18258,2020-12-29,동작구,nan,감염경로 조사중,퇴원,이동경로 공개기간 경과
...,...,...,...,...,...,...,...
4,18855,2020-12-30,종로구,nan,감염경로 조사중,사망,이동경로 공개기간 경과
3,18856,2020-12-30,기타,nan,동부구치소 관련,nan,이동경로 공개기간 경과
2,18857,2020-12-30,기타,nan,동부구치소 관련,nan,이동경로 공개기간 경과
1,18858,2020-12-30,중랑구,nan,기타 확진자 접촉,퇴원,이동경로 공개기간 경과


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 605 to 0
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      606 non-null    int32 
 1   확진일     606 non-null    object
 2   지역      606 non-null    object
 3   여행력     5 non-null      object
 4   접촉력     606 non-null    object
 5   상태      414 non-null    object
 6   이동경로    606 non-null    object
dtypes: int32(1), object(6)
memory usage: 55.5+ KB


In [91]:
df['확진일'] = pd.to_datetime(df['확진일'])
df

TypeError: Argument 'date_string' has incorrect type (expected str, got NavigableString)

https://yhdevlog.tistory.com/entry/Pandas-%EC%9B%B9-%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91-%EB%8D%B0%EC%9D%B4%ED%84%B0-todatetime-%ED%98%95%EB%B3%80%ED%99%98-%EC%98%A4%EB%A5%98

In [ ]:
df['지역'].fillna('기타', inplace=True)
df['지역'] = df['지역'].map(lambda s: s.strip())
df.fillna(' ', inplace=True)